In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
task_names = [
    "COLA", 
#     "SST2",
#     "MNLI",
#     "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]

FILENAME = "COLA_tasks_and_payloads"

In [3]:
SEED = 1

## Baseline: Train COLA Task

In [4]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# Create tasks and payloads
tasks, payloads = create_tasks_and_payloads(
    task_names,
    dl_kwargs={"batch_size": 16},
    freeze_bert=True
)

Using random seed: 387965
Loading COLA Dataset



CPU times: user 13.7 s, sys: 6.12 s, total: 19.8 s
Wall time: 15 s


In [5]:
tasks, payloads

([ClassificationTask(name=COLA, loss_multiplier=1.0)],
 [Payload(COLA_train: tasks=[COLA], split=train),
  Payload(COLA_valid: tasks=[COLA], split=valid),
  Payload(COLA_test: tasks=[COLA], split=test)])

In [6]:
from metal.mmtl.metal_model import MetalModel

model_baseline = MetalModel(tasks, seed=SEED, verbose=False)

In [7]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

In [8]:
# trainer.train_model(
#     model_baseline,
#     payloads,
#     checkpoint_metric="COLA/COLA_valid/matthews_corr",
#     checkpoint_metric_mode="max",
#     writer="tensorboard",
#     optimizer="adamax",
#     lr=5e-5,
#     log_every=0.05,
#     score_every=0.1,
#     n_epochs=3,
#     progress_bar=True
# )

In [9]:
model_baseline.score(payloads[1])

1042 1042


/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/site-packages/sklearn/metrics/classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


{'COLA/COLA_valid/accuracy': 0.690978886756238,
 'COLA/COLA_valid/matthews_corr': 0.0}

## Slice-aware Model

In [10]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# define slices
slice_dict = {  # A map of the slices that apply to each task
   "COLA": ["ends_with_question_mark"]
}

# Create tasks and payloads
tasks_slice, payloads_slice = create_tasks_and_payloads(
    task_names,
#     max_datapoints=1000,
    dl_kwargs={"batch_size": 16},
    slice_dict=slice_dict,
    freeze_bert=True
)

Using random seed: 128037
Loading COLA Dataset



Added label_set with 8550 labels for task COLA:ends_with_question_mark to payload COLA_train.
Added label_set with 1042 labels for task COLA:ends_with_question_mark to payload COLA_valid.
Added label_set with 1063 labels for task COLA:ends_with_question_mark to payload COLA_test.
CPU times: user 8.92 s, sys: 964 ms, total: 9.89 s
Wall time: 10.9 s


### Evaluate baseline model on the slice of interest

In [11]:
# Allow us to evaluate on arbitrary payload with arbitrary model
model_baseline.score(payloads_slice[1])

1042 1042
57 1042


{'COLA/COLA_valid/accuracy': 0.690978886756238,
 'COLA/COLA_valid/matthews_corr': 0.0,
 'COLA:ends_with_question_mark/COLA_valid/accuracy': 0.6140350877192983,
 'COLA:ends_with_question_mark/COLA_valid/matthews_corr': 0.0}

In [12]:
# tasks_slice, payloads_slice

In [13]:
import numpy as np
def count_num_labels(labels):
    return np.sum(np.array(labels) != 0)

In [14]:
dataset = payloads_slice[0].data_loader.dataset
for labelset_name, labels in dataset.labels.items():
    print(labelset_name, count_num_labels(labels))

COLA 8550
COLA:ends_with_question_mark 615


In [15]:
from metal.mmtl.metal_model import MetalModel

model_slice = MetalModel(tasks_slice, seed=SEED, verbose=False)

In [16]:
from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

In [17]:
# trainer.train_model(
#     model_slice,
#     payloads_slice,
#     checkpoint_metric="COLA/COLA_valid/matthews_corr",
#     checkpoint_metric_mode="max",
#     writer="tensorboard",
#     optimizer="adamax",
#     lr=5e-5,
#     log_every=0.05,
#     score_every=0.1,
#     n_epochs=3,
#     progress_bar=True
# )

In [18]:
payloads_slice[0].task_names

['COLA', 'COLA:ends_with_question_mark']

In [19]:
from metal.mmtl.metal_model import MetalModel
model_slice.score(payloads_slice[1])

1042 1042
57 1042


{'COLA/COLA_valid/accuracy': 0.690978886756238,
 'COLA/COLA_valid/matthews_corr': 0.0,
 'COLA:ends_with_question_mark/COLA_valid/accuracy': 0.6140350877192983,
 'COLA:ends_with_question_mark/COLA_valid/matthews_corr': 0.0}

In [20]:
print("WE WIN!")

WE WIN!
